In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from IPython.display import display
import keras
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.layers import BatchNormalization, GlobalAveragePooling2D, Dense, Activation, Dropout
from keras.layers import MaxPooling2D, Conv2D, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam, SGD
from keras import regularizers
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import time
from tqdm import tqdm
import gc
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
% matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../input/dl3-dataset/meta-data/meta-data/"))

# Any results you write to the current directory are saved as output.

# Steps

1. Load the given training data [Images, Attributes]
2. Preprocess the images [resize, rescale]
3. Use Transfer Learning with VGG19


In [2]:
train_data = pd.read_csv('../input/dl3-dataset/meta-data/meta-data/train.csv')
train_data.head()

In [3]:
classes = pd.read_csv('../input/dl3-dataset/classes.txt', header=None)
classes.head()

In [4]:
att = pd.read_csv('../input/dl3-dataset/attributes.txt', header=None)
att.head()

In [5]:
label_cols = list(set(train_data.columns) - set(['Image_name']))
label_cols.sort()

In [6]:
i = random.randint(1, len(train_data))
im = Image.open(os.path.join('../input/dl3-dataset/train_img/train_img', train_data.Image_name[i]))
labels = train_data.iloc[i][2:].index[train_data.iloc[i][2:] == 1]
plt.imshow(im)
print (im.size)
print (labels)

In [7]:
# def prep_data(x):
#     d = []
#     for i in x:
#         img = cv2.imread(os.path.join('../input/dl3-dataset/train_img/train_img', i))
#         img = cv2.resize(img, (128, 128))
#         img = img / 255.
#         d.append(img)
#     return np.array(d, np.float32)

def prep_data(x_d):
    d = []
    for i in tqdm(x_d):
        img = image.load_img(os.path.join('../input/dl3-dataset/train_img/train_img', i), target_size=(128, 128))
        x = image.img_to_array(img)
        x = x / 255.
        x = np.expand_dims(x, axis=0)
        x = np.squeeze(preprocess_input(x))
        d.append(x)
    return np.array(d)

In [8]:
data = prep_data(train_data.Image_name.values)
display(data.shape)

In [9]:
labels = train_data.drop(['Image_name'], axis=1)
display(labels.head())
labels = np.array(labels)
display(labels.shape)

In [10]:
train_x,val_x, train_y, val_y = train_test_split(data, labels, test_size=0.2, shuffle=True)
print ('Training shape', train_x.shape, train_y.shape)
print ('Validation shape', val_x.shape, val_y.shape)

In [11]:
del train_data, data, labels
gc.collect()

In [12]:
batch_size = 32
num_epochs = 25
num_classes = len(att)
iters_train = train_x.shape[0] / batch_size
iters_val = val_x.shape[0] / batch_size

In [21]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(width_shift_range=0.2, height_shift_range=0.2,
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True, rotation_range = 20)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [22]:
train_datagen.fit(train_x)
val_datagen.fit(val_x)

In [ ]:
#only needed for Kaggle Kernel, locally just use weights='imagenet'
vgg19_fl = "../input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"

base_model = VGG19(weights=vgg19_fl, include_top=False, input_shape=(128, 128, 3))
# Freeze the layers in base model
for i, layer in enumerate(base_model.layers):
    layer.trainable = False
    
x_i = base_model.output
x_i = GlobalAveragePooling2D()(x_i)
x_i = Dense(200)(x_i)
x_i = BatchNormalization()(x_i)
x_i = Activation('relu')(x_i)
x_i = Dropout(0.5)(x_i)
# x_i = Dense(512)(x_i)
# x_i = BatchNormalization()(x_i)
# x_i = Activation('relu')(x_i)
# x_i = Dropout(0.5)(x_i)
predictions = Dense(num_classes, activation='sigmoid')(x_i)

model = Model(inputs=base_model.input, outputs=predictions)
model.summary()
opt = Adam(lr=1e-3, decay=1e-6)
filepath = 'model_vgg19.hdf5'
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])
mc = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, period=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
# history = model.fit(train_x, train_y, validation_data=(val_x, val_y), batch_size=batch_size, epochs=4, verbose=1)
model.fit_generator(train_datagen.flow(train_x, train_y, batch_size=batch_size),                     
                    steps_per_epoch=iters_train, epochs=5,
                    validation_data=val_datagen.flow(val_x, val_y, batch_size=batch_size), 
                    validation_steps = iters_val, callbacks=[early_stopping, mc], workers = 10, max_queue_size=20)

In [ ]:
for i, layer in enumerate(model.layers[:17]):
    layer.trainable = False

for i, layer in enumerate(model.layers[17:]):
    layer.trainable = True

opt = SGD(lr=0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
filepath = 'model_vgg19_finetune.hdf5'
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])
mc = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, period=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
# history = model.fit(train_x, train_y, validation_data=(val_x, val_y), batch_size=batch_size, epochs=num_epochs, callbacks=[early_stopping, mc], verbose=1)
# model.fit_generator(train_generator(), steps_per_epoch=iters_train, epochs=num_epochs, 
#                     verbose=1, validation_data=valid_generator(), validation_steps=iters_val)
model.fit_generator(train_datagen.flow(train_x, train_y, batch_size=batch_size),                     
                    steps_per_epoch=iters_train, epochs=5,
                    validation_data=val_datagen.flow(val_x, val_y, batch_size=batch_size), 
                    validation_steps = iters_val, callbacks=[early_stopping, mc], workers = 10, max_queue_size=20)

In [17]:
model = Sequential()
model.add(BatchNormalization(input_shape=(128, 128, 3)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(Conv2D(50, kernel_size=(3, 3), padding='same', kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(BatchNormalization())
model.add(Conv2D(50, kernel_size=(3, 3), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(100, kernel_size=(3, 3), padding='same', kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(BatchNormalization())
model.add(Conv2D(100, kernel_size=(3, 3), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(85, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))

# model = Model(inputs=base_model.input, outputs=predictions)
model.summary()
opt = Adam(lr=1e-3, decay=1e-6)
filepath = 'model_custom.hdf5'
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])
mc = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, period=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
# history = model.fit(train_x, train_y, validation_data=(val_x, val_y), batch_size=batch_size, epochs=num_epochs, verbose=1)
# model.fit_generator(train_generator(), steps_per_epoch=iters_train, epochs=num_epochs, 
#                     verbose=1, validation_data=valid_generator(), validation_steps=iters_val)
model.fit_generator(train_datagen.flow(train_x, train_y, batch_size=batch_size),                     
                    steps_per_epoch=iters_train, epochs=25,
                    validation_data=val_datagen.flow(val_x, val_y, batch_size=batch_size), 
                    validation_steps = iters_val, callbacks=[early_stopping, mc], workers = 10, max_queue_size=20)

In [18]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [19]:
start = time.time()
val_pred = model.predict(val_x).round()
print ('Time for prediction', time.time()-start)
print (val_pred[0], val_pred[0].shape)
print (val_y[0], val_y[0].shape)
print (f1_score(val_y, val_pred, average='samples'))

In [ ]:
test_data = pd.read_csv('../input/meta-data/meta-data/test.csv')
test_data.head()

In [ ]:
test_x = prep_data(test_data)
sub = pd.DataFrame(columns=[''])
sub['Image_name'] = test_data.Image_name
del test_data
gc.collect()

In [ ]:
start = time.time()
test_pred = model.predict(test_x).round()
print ('Time for prediction', time.time()-start)

In [ ]:
label_df = pd.DataFrame(data=test_pred, columns=label_cols)
sub = pd.concat([sub, label_df], axis=1)
sub.to_csv('submit.csv', index=False)